In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [181]:
#export
from exp.nb_11 import *

We're gonna try to train a model pretrained on the imagewoof dataset on the cats and dogs dataset from Lesson1.

## Serializing the model

In [16]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160)

In [168]:
size = 128
bs = 128

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y = CategoryProcessor())
ll.valid.x.tfms= val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [18]:
len(il)

12954

In [19]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [20]:
train_dl, _ = get_dls(ll.train,ll.valid,bs, num_workers=4)
x, _ = next(iter(train_dl))
# x.shape
m ,s  = x.mean((0,2,3)).cuda(), x.std((0,2,3)).cuda()
m,s

(tensor([0.4914, 0.4409, 0.3674], device='cuda:0'),
 tensor([0.2484, 0.2343, 0.2409], device='cuda:0'))

In [21]:
_m = tensor([0.49, 0.46, 0.40])
_s = tensor([0.25, 0.24, 0.25])
norm_imagewoof = partial(normalize_chan, mean=_m.cuda(), std=_s.cuda())

In [27]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagewoof, mixup=0.2)

In [114]:
# ??cnn_learner

In [29]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
           ParamScheduler('mom', sched_mom)]

In [30]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [31]:
cbsched

In [32]:
learn.fit(60, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.208265,0.222715,2.103373,0.246882,00:11
1,2.108967,0.279557,2.039489,0.293204,00:10
2,2.046957,0.324986,2.043359,0.282260,00:10
3,2.006238,0.351911,1.923007,0.350471,00:10
4,1.959454,0.380942,1.941493,0.349198,00:10
5,1.908258,0.402216,1.924641,0.378977,00:11
6,1.886229,0.423823,1.766721,0.438534,00:11
7,1.848982,0.446981,1.797074,0.422754,00:10
8,1.839771,0.452632,1.753295,0.435225,00:11
9,1.804731,0.473241,1.983871,0.386103,00:11


In [34]:
learn.fit(40, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.078665,0.893961,0.972883,0.813693,00:11
1,1.081557,0.890748,0.983259,0.813693,00:10
2,1.082282,0.894737,0.987209,0.808857,00:10
3,1.077644,0.892964,0.988271,0.805294,00:10
4,1.080959,0.888643,0.979539,0.812675,00:10
5,1.089878,0.884765,1.022963,0.796131,00:11
6,1.085915,0.883657,1.032022,0.785951,00:11
7,1.103616,0.883324,1.080189,0.764062,00:10
8,1.118001,0.869806,1.049888,0.780351,00:11
9,1.123964,0.864377,1.046463,0.785187,00:10


In [35]:
st = learn.model.state_dict()

In [36]:
type(st)

collections.OrderedDict

In [37]:
','.join(st.keys())

'0.0.weight,0.1.weight,0.1.bias,0.1.running_mean,0.1.running_var,0.1.num_batches_tracked,1.0.weight,1.1.weight,1.1.bias,1.1.running_mean,1.1.running_var,1.1.num_batches_tracked,2.0.weight,2.1.weight,2.1.bias,2.1.running_mean,2.1.running_var,2.1.num_batches_tracked,4.0.convs.0.0.weight,4.0.convs.0.1.weight,4.0.convs.0.1.bias,4.0.convs.0.1.running_mean,4.0.convs.0.1.running_var,4.0.convs.0.1.num_batches_tracked,4.0.convs.1.0.weight,4.0.convs.1.1.weight,4.0.convs.1.1.bias,4.0.convs.1.1.running_mean,4.0.convs.1.1.running_var,4.0.convs.1.1.num_batches_tracked,4.1.convs.0.0.weight,4.1.convs.0.1.weight,4.1.convs.0.1.bias,4.1.convs.0.1.running_mean,4.1.convs.0.1.running_var,4.1.convs.0.1.num_batches_tracked,4.1.convs.1.0.weight,4.1.convs.1.1.weight,4.1.convs.1.1.bias,4.1.convs.1.1.running_mean,4.1.convs.1.1.running_var,4.1.convs.1.1.num_batches_tracked,5.0.convs.0.0.weight,5.0.convs.0.1.weight,5.0.convs.0.1.bias,5.0.convs.0.1.running_mean,5.0.convs.0.1.running_var,5.0.convs.0.1.num_batches_tra

In [38]:
st['10.bias']

tensor([-0.0090,  0.0168,  0.0262, -0.0154, -0.0459, -0.0081, -0.0015,  0.0095,
         0.0071,  0.0084], device='cuda:0')

In [39]:
len(st)

134

In [40]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

It's also possible to save the whole model, including the architecture, but it gets quite fiddly and they don't recommend it. Instead, just save the parameters and recreate the model directly. 

In [41]:
torch.save(st, mdl_path/'iw5')
# you can also use pickle

In [42]:
path

PosixPath('/home/jupyter/.fastai/data/imagewoof2-160')

### Pets

In [227]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [228]:
pets.ls()

[PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/old'),
 PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/image_gen'),
 PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images'),
 PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/crappy'),
 PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/models'),
 PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/annotations')]

In [229]:
pets_path = pets/'images'

In [230]:
il = ImageList.from_files(pets_path, tfms = tfms)

In [231]:
il

ImageList (7390 items)
[PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/pomeranian_123.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Bengal_87.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Ragdoll_52.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_54.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/samoyed_165.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_87.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Sphynx_189.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Bengal_116.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Siamese_185.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/staffordshire_bull_terrier_93.jpg')...]
Path: /home/jupyter/.fastai/data/oxford-iiit-pet/images

In [232]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [233]:
random.seed(30)

In [234]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [235]:
sd

SplitData
Train: ImageList (6665 items)
[PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/pomeranian_123.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Bengal_87.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_54.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/samoyed_165.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_87.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Sphynx_189.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Bengal_116.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/Siamese_185.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/staffordshire_bull_terrier_93.jpg'), PosixPath('/home/jupyter/.fastai/data/oxford-iiit-pet/images/pomeranian_9.jpg')...]
Path: /home/jupyter/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (725 items)
[PosixPath('/home/jupyter/.fastai/data/oxf

In [236]:
n = il.items[0].name; n

'pomeranian_123.jpg'

In [237]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'pomeranian'

In [238]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [239]:
proc = CategoryProcessor()

In [240]:
ll = label_by_func(sd, pet_labeler, proc_y = proc)

In [241]:
# ??LabeledData

In [242]:
','.join(proc.vocab)

'pomeranian,Bengal,wheaten_terrier,samoyed,Egyptian_Mau,Sphynx,Siamese,staffordshire_bull_terrier,basset_hound,chihuahua,British_Shorthair,Maine_Coon,Ragdoll,keeshond,Abyssinian,newfoundland,Russian_Blue,japanese_chin,saint_bernard,beagle,great_pyrenees,boxer,english_setter,havanese,Bombay,american_bulldog,english_cocker_spaniel,yorkshire_terrier,american_pit_bull_terrier,Birman,shiba_inu,scottish_terrier,german_shorthaired,leonberger,pug,miniature_pinscher,Persian'

In [243]:
ll.valid.x.tfms = val_tfms

In [244]:
c_out = len(proc.vocab); c_out

37

In [245]:
bs

128

In [246]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [247]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagewoof)

In [248]:
# witout using Transfer Learning
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.501664,0.081920,3.451468,0.082759,00:09
1,3.352349,0.121230,3.518072,0.089655,00:10
2,3.180913,0.164441,3.599262,0.089655,00:09
3,2.939767,0.228357,2.898027,0.220690,00:10
4,2.696167,0.306977,2.690335,0.311724,00:10


## Custom Head

In [249]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette, mixup=0.2)

In [250]:
st = torch.load(mdl_path/'iw5')

In [251]:
m = learn.model

In [252]:
m

XResNet(
  (0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (1): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): ResBlock(
      (convs): Sequential(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine

In [253]:
m.load_state_dict(st)

<All keys matched successfully>

In [254]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [255]:
cut

8

In [256]:
xb, yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [257]:
pred = m_cut(xb)
pred.shape

torch.Size([256, 512, 4, 4])

In [258]:
ni = pred.shape[1]; ni

512

In [259]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
        
    def forward(self, x):
        return torch.cat([self.mp(x), self.ap(x)],1)

In [260]:
nh = 40 

m_new = nn.Sequential(m_cut, 
                     AdaptiveConcatPool2d(), Flatten(),
                     nn.Linear(ni*2, data.c_out))
# ni*2 because we're concatenating in the AdaptivePool step. thus i/p would be 1024 features

In [261]:
learn.model = m_new
learn.model = learn.model.cuda()

In [262]:
??BatchTransformXCallback

Init signature: BatchTransformXCallback(tfm)
Docstring:      <no docstring>
Source:        
class BatchTransformXCallback(Callback):
    _order=2
    def __init__(self, tfm): self.tfm = tfm
    def begin_batch(self): self.run.xb = self.tfm(self.xb)
File:           ~/tutorials/fastai/course-v3/nbs/dl2/exp/nb_06.py
Type:           type
Subclasses:     


In [263]:
model_summary(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


 torch.Size([256, 512, 4, 4])
 torch.Size([256, 1024, 1, 1])
 torch.Size([256, 1024])
 torch.Size([256, 37])


In [264]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.250581,0.190698,2.362771,0.427586,00:10
1,2.583269,0.424306,2.421825,0.351724,00:10
2,2.390991,0.504276,2.064920,0.503448,00:10
3,2.222934,0.573893,1.700859,0.668966,00:09
4,2.099585,0.641410,1.586322,0.707586,00:10


And that's the power of transfer learning!

## Freezing and unfreezing

In [265]:
#  Refactoring part of above code into a function
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(m.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
    m_cut = m[:cut]
    xb, yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(m_cut, 
                     AdaptiveConcatPool2d(), Flatten(),
                     nn.Linear(ni*2, data.c_out))
    learn.model = m_new

In [266]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagewoof, mixup=0.2)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

<All keys matched successfully>

In [267]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


To freeze the pretrained part of model, we simply set its required_grad to False

In [268]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [269]:
learn.fit(3, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.933616,0.363991,1.795942,0.630345,00:09
1,2.212320,0.590848,1.646337,0.664828,00:09
2,2.106514,0.633008,1.593808,0.711724,00:09


In [270]:
for p in learn.model[0].parameters(): p.requires_grad_(True)

In [271]:
learn.fit(3, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.091927,0.644261,1.773063,0.637241,00:10
1,2.150088,0.610203,1.778141,0.584828,00:10
2,2.035712,0.659415,1.514877,0.713103,00:10


Turns out that sometimes freezing and unfreezing like that can get us worse results than training the whole pretrained model for a few epochs. This is mainly because of BatchNorm. What happens is that the freezed part of model had weights which were tuned to some specific to the means and stds( the batchnorm divides by the std and subracts the mean), but this new dataset might have different means and stds(not necessarily for the input but inside the model). So after freezing the final layers have weights specific to the BN statistics in the initial layers and after unfreezing everything starts to catch up and kinda ends up going haywire. 

Research idea: plot the histograms and stuff done previously and try inferring what is happening.

## BatchNorm Transfer

In [272]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [273]:
def apply_mod(m, f):
    f(m)
    for l in m.children(): apply_mod(l, f)
        
def set_grad(m, b):
    if isinstance(m, (nn.Linear, nn.BatchNorm2d)): return # don't do anything if a batchnorm  or Linear layer is present
    if hasattr(m, 'weight'):
        for p in m.parameters(): p.requires_grad_(b) # so for all conv layers

In [274]:
apply_mod(learn.model, partial(set_grad, b=False))

In [275]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.183096,0.540285,1.805121,0.613793,00:09
1,1.575055,0.696924,1.562432,0.724138,00:09
2,1.458325,0.743736,1.454348,0.732414,00:09


In [276]:
apply_mod(learn.model, partial(set_grad, b=True))

In [277]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.404136,0.758740,1.445292,0.748966,00:10
1,1.412181,0.749437,1.599928,0.692414,00:10
2,1.472194,0.726932,1.762474,0.620690,00:10
3,1.402360,0.759340,1.528425,0.710345,00:10
4,1.249877,0.825056,1.370125,0.755862,00:09


Pytorch already has an `apply` method we can use

In [278]:
learn.model.apply(partial(set_grad, b=False));

## Discriminative LR and param groups

One way to use Discriminative LRs without worrying about the BN part in freeze/unfreeze is to set the lr = 0 for some layers

In [279]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [280]:
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
        elif hasattr(l, 'weight'): g1 += l.parameters()
        for ll in l.children(): _bn_splitter(ll, g1, g2)
        
    g1, g2 = [], []
    _bn_splitter(m[0], g1, g2)
    
    g2 += m[1:].parameters()
    return g1, g2


In [281]:
a, b = bn_splitter(learn.model)

In [282]:
test_eq(len(a)+ len(b), len(list(m.parameters())))

In [283]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [285]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [287]:
cb_types

namespace(after_backward='after_backward', after_batch='after_batch', after_cancel_batch='after_cancel_batch', after_cancel_epoch='after_cancel_epoch', after_cancel_train='after_cancel_train', after_epoch='after_epoch', after_fit='after_fit', after_loss='after_loss', after_pred='after_pred', after_step='after_step', begin_batch='begin_batch', begin_epoch='begin_epoch', begin_fit='begin_fit', begin_validate='begin_validate')

In [286]:
cb_types.after_backward

'after_backward'

How do we know if it's working cause we probably won't get an error here and instead some layer wouldn't get trained resulting in poor accuracy. Ofcourse we use a Callback to do this and override the original dunder __call__ method of the Callback class. We use the _print_det function below to print the two param groups and and exit training.

In [288]:
#export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name, self.f = cb_name, f
    
    # overriding dunder call of the parent class here.
    def __call__(self, cb_name):
        if cb_name == self.cb_name:
            if self.f: self.f(self.run)
            else: set_trace()

In [291]:
# export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
               for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
           ParamScheduler('mom', sched_mom)]

In [292]:
disc_lr_sched = sched_1cycle([0,3e-2], 0.5)

In [293]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func,
                    c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [294]:
def _print_det(o):
    print(len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()
    
learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


lr is 0.003 because we're using ParamScheduler for learning rates and hence it starts at lr/10

In [295]:
learn.fit(3, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.804369,0.657614,2.123167,0.544828,00:09
1,1.548525,0.710728,1.683707,0.668966,00:10
2,1.418305,0.756789,1.422037,0.748966,00:10


In [296]:
disc_lr_sched = sched_1cycle([1e-3,1e-2], 0.3)

In [297]:
learn.fit(5, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.306746,0.803001,1.442650,0.743448,00:09
1,1.316976,0.793698,1.526641,0.715862,00:09
2,1.309250,0.795349,1.403733,0.755862,00:09
3,1.250302,0.824006,1.365287,0.769655,00:09
4,1.198872,0.839310,1.372510,0.762759,00:09


Nice!

## Export

In [ ]:
!./notebook2script.py 11a_transfer_learning.ipynb